In [180]:
!pip install datasets
!pip install transformers
!pip install evaluate   
!pip install transformers[torch]
!pip install accelerate -U

In [181]:
import pandas as pd
import tensorflow as tf
import numpy as np

In [182]:
df = pd.read_csv('spam_train.csv', encoding='latin-1')
df.drop(columns=['Unnamed: 0'],inplace= True)


In [183]:
df.sample(5)

,message_type,message
323,1,Ur cash-balance is currently 500 pounds - to m...
2332,0,SO IS TH GOWER MATE WHICH IS WHERE I AM!?! HOW...
2817,0,"I'll get there at 3, unless you guys want me t..."
3211,0,I told your number to gautham..
686,0,Same to u...


In [184]:
df['message_type'].value_counts()

message_type
0    4299
1     611
Name: count, dtype: int64

In [185]:
# check percentage of data - states how much data needs to be balanced
print(str(round(611/4299,2)*100)+'%')

14.000000000000002%


In [186]:
df_spam = df[df['message_type']==1]
df_ham = df[df['message_type']==0]
print("Ham Dataset Shape:", df_ham.shape)
print("Spam Dataset Shape:", df_spam.shape)

Ham Dataset Shape: (4299, 2)
Spam Dataset Shape: (611, 2)


In [187]:
# downsampling ham dataset - take only random 747 example
# will use df_spam.shape[0] - 747
df_ham_downsampled = df_ham.sample(df_spam.shape[0])
df_ham_downsampled.shape

(611, 2)

In [188]:
# concating both dataset - df_spam and df_ham_balanced to create df_balanced dataset
df_balanced = pd.concat([df_spam , df_ham_downsampled])

In [189]:
df_balanced['message_type'].value_counts()

message_type
1    611
0    611
Name: count, dtype: int64

In [190]:
df_balanced.sample(10)

,message_type,message
2076,0,"Sorry chikku, my cell got some problem thts y ..."
2180,0,Not thought bout it... || Drink in tap & spile...
847,1,Knock Knock Txt whose there to 80082 to enter ...
4707,0,How much i gave to you. Morning.
4426,0,2marrow only. Wed at &lt;#&gt; to 2 aha.
1547,1,1000's flirting NOW! Txt GIRL or BLOKE & ur NA...
2102,1,"Loan for any purpose Ã¥Â£500 - Ã¥Â£75,000. Hom..."
1574,0,Jesus armand really is trying to tell everybod...
4486,0,K..then come wenever u lik to come and also te...
3374,1,HOT LIVE FANTASIES call now 08707509020 Just 2...


In [191]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

In [192]:
from transformers import AutoModelForSequenceClassification
model = AutoModelForSequenceClassification.from_pretrained('bert-base-uncased',num_labels=2)

In [193]:
from datasets import Dataset
train_data_df = df_balanced.sample(200, random_state=42)
eval_data_df = df_balanced.sample(200, random_state=45)
train_dataset = Dataset.from_pandas(train_data_df)
eval_dataset = Dataset.from_pandas(eval_data_df)
test_data_df = df_balanced.iloc[4000:4100]
test_dataset = Dataset.from_pandas(test_data_df)

In [194]:
def tokenize_function(examples):
    return tokenizer(examples["message"], padding="max_length", truncation=True)
tokenized_datasets_train = train_dataset.map(tokenize_function, batched=True)
tokenized_datasets_eval = eval_dataset.map(tokenize_function, batched=True)
tokenizer_datasets_test = test_dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

In [195]:
from transformers import TrainingArguments,Trainer
training_args = TrainingArguments(output_dir="test_trainer")


ImportError: Using the `Trainer` with `PyTorch` requires `accelerate>=0.21.0`: Please run `pip install transformers[torch]` or `pip install accelerate -U`